In [27]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import keras
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Sequential
from keras.models import load_model

In [28]:
def load_data(input_size = (48,48), data_path = 'C:\\Users\LEGION\OneDrive\Desktop\train'):

    pixels = []
    labels = []
    
    # Loop qua các thư mục trong thư mục Train
    i=0
    for dir in os.listdir(data_path):
        class_dir = os.path.join(data_path, dir)
        # Lăp trong file
            # Đọc ảnh
            #pixel = imread(os.path.join(class_dir, )
            # Resize về kích cỡ chuẩn
            #img = cv2.resize(pixel, input_size)
            # Thêm vào list dữ liệu
            #pixels.append(img)
            # Thêm nhãn cho ảnh
            #labels.append(dir)
        path = glob.glob(class_dir + '\*.jpg')
        for img in path:
            labels.append(i)
            pixel = cv2.imread(img)
            pixels.append(pixel)
        i+=1
    return pixels, labels

In [29]:
import glob
data_path = 'C:\\Users\LEGION\OneDrive\Desktop\\train'
pixels, labels = load_data(data_path=data_path)

In [33]:
def split_train_val_test_data(pixels, labels):

    # Chuẩn hoá dữ liệu pixels và labels
    pixels = np.array(pixels)
    labels = keras.utils.np_utils.to_categorical(labels)

    # Nhào trộn dữ liệu ngẫu nhiên
    randomize = np.arange(len(pixels))
    np.random.shuffle(randomize)
    X = pixels[randomize]
    print("X=", X.shape)
    y = labels[randomize]

    # Chia dữ liệu theo tỷ lệ 60% train và 40% còn lại cho val và test
    train_size = int(X.shape[0] * 0.6)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    val_size = int(X_val.shape[0] * 0.5) # 50% của phần 40% bên trên
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_val, y_test = y_val[:val_size], y_val[val_size:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [34]:
X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test_data(pixels, labels)

X= (28709, 48, 48, 3)


In [35]:
def build_model(input_shape=(48,48,3), filter_size = (3,3), pool_size = (2, 2), output_size = 48):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,3)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
    model.summary()
    return model

In [36]:
model = build_model(input_shape=(48,48,3), output_size=48)

# Train model
epochs = 30
batch_size = 10

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                               validation_data=(X_val, y_val))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 46, 46, 32)        896       
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 8, 8, 128)        

In [21]:
model.save("emotion_model.h5")